<a href="https://colab.research.google.com/github/tmckim/materials-fa23-colab/blob/main/lectures/lec06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**Copy of lec06.ipynb**". You can rename this to just the title of the assignment "**lec06.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them.

In [ ]:
# Setup and add files needed to gdrive
# If you restart colab, start by rerunning this cell first!
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#!mkdir -p '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/'
%cd /content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/
!rm -r materials-fa23-colab

!git clone https://github.com/tmckim/materials-fa23-colab '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/'

%cd /content/gdrive/MyDrive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/lectures/

In [ ]:
# Import packages and other things needed
# Don't change this cell; Just run this cell
# If you restart colab, make sure to run this cell again after the first one above^

from datascience import *
import numpy as np
# path_data = '../../../assets/data/' ######### is this needed?

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

## Lecture 06 ##


Topics:
- Reviewing table methods like join, group, pivot and apply
- conditional statements
- for loops
- using `apply` for rows
- simulation and using `np.random.choice`

## A Join Example ##

In [ ]:
# One table
full = Table.read_table('nc-est2019-agesex-res.csv')
census = full.select('SEX', 'AGE', 'POPESTIMATE2019')
census.show(3)

In [ ]:
# A second table
sex_codes = Table().with_columns(
    'SEX CODE', make_array(0, 1, 2),
    'CODE DEFINITION', make_array('All', 'Selected Male', 'Selected Female')
)
sex_codes

In [ ]:
# Join the tables
sex_codes.join('SEX CODE', census, 'SEX').sort('AGE').show(3)

## Bike Sharing ##

In [ ]:
# Read in the trip data from csv file
trip = Table.read_table('trip.csv')
trip.show(3)

## Distribution of Durations ##

In [ ]:
# Visualize the durations (seconds)
trip.hist('Duration')

Why does the histogram look like this?

In [ ]:
# Sort by duration with largest values first
trip.sort('Duration', descending=True)

In [ ]:
# Just trips under 30 mins (free ride length)
commute = trip.where('Duration', are.below(60*30)) # 60 secs * 30 mins
commute.hist('Duration')

Why are there not any values right after 0?<br>Hint: there was a minimum time limit

In [ ]:
# Now specify bins instead of letting hist automatically pick it
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

Based on the histogram, can you approximate the percent of people who have a ride duration between 250 and 500 seconds? <br>
Hint: Think about what formula you need and what information you have.

In [ ]:
# Approx percent of people who have
# a ride duration between 250 and 500 seconds
# "between" = [250, 500)

(500-250) * 0.15

Calculating the exact value:

In [ ]:
# Get the number of rows using .where
commute.where('Duration', are.between(250, 500)).num_rows

In [ ]:
# take number of trips from the duration window and divide by the number of rows in the table
# gives us percentage in that interval
129079 / commute.num_rows * 100

In [ ]:
# Same code as above
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
 # Now put in a number of bins for comparison
commute.hist('Duration', bins=60, unit='Second')

## Start and End Stations ##

In [ ]:
# Most common start station
# group finds unique values of the Start Station and gets the count of those
starts = commute.group('Start Station').sort('count', descending=True)
starts

What could we use to look at trips between stations?

In [ ]:
# Using group
commute.group(['Start Station', 'End Station'])

In [ ]:
# Using pivot

commute.pivot('Start Station', 'End Station')

Why are there so many zeros? Anything you remember about what portion of the dataset we are using?

Let's look at the average duration for the combination of the stations?

In [ ]:
# Average durations of trips between stations
commute.pivot('Start Station', 'End Station', values='Duration', collect=np.average)

In [ ]:
# Time it should take- direct without stopping
commute.pivot('Start Station', 'End Station', values='Duration', collect=min)

What if we wanted to put multiple numbers in our table that include the min and the max?

In [ ]:
# Define our own function and test it out
def min_and_max(values):
  return str(min(values)) + "," + str(max(values))

In [ ]:
# Time it should take- direct without stopping
commute.pivot('Start Station', 'End Station', values='Duration', collect=min_and_max)

How can we find the fastest trip ever between each pair of stations using group?

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
shortest = duration.group(['Start Station', 'End Station'], min)
shortest.show(5)

## Discussion question

Find the 5 stations closest to Civic Center BART by minimum trip time.

In [ ]:
from_cc = shortest.where(
    'Start Station', are.containing('Civic Center BART')).sort(
    'Duration min')
from_cc.take(np.arange(5)).column('End Station')

## Maps ##

In [ ]:
# Geographical data on the stations (dropping some columns we don't need)
stations = Table.read_table('station.csv').drop(4, 6)
stations

In [ ]:
sf_stations = stations.where('landmark', are.equal_to('San Francisco'))
sf_stations_map_data = (sf_stations
 .select('lat', 'long', 'name')
 .relabeled('name', 'labels'))
sf_stations_map_data.show(3)

In [ ]:
Marker.map_table(sf_stations_map_data)

In [ ]:
Circle.map_table(sf_stations_map_data)

### Optional: Discussion question

Map all stations within 4 minutes (minimum ride time) of Civic Center.

In [ ]:
from_cc.show(3)

In [ ]:
sf_stations_map_data.show(3)

In [ ]:
close_cc = from_cc.where('Duration min', are.below(4 * 60))
close_markers = (sf_stations_map_data
      .join('labels', close_cc, 'End Station')
      .select('lat', 'long', 'labels'))
Marker.map_table(close_markers)

Choose marker colors by the minimum time from Civic Center

In [ ]:
minutes = np.round(from_cc.column("Duration min") / 60)
print(min(minutes), max(minutes))

In [ ]:
colors = Table().with_columns(
    "minutes", np.arange(15),
    "colors",  ["darkblue", "blue", "lightblue",
                "darkgreen", "green", "lightgreen",
                "orange", "darkred", "red",
                "gray", "gray", "gray",
                "gray", "gray", "gray"])
colors_cc = (from_cc
 .with_column("Minutes", minutes)
 .join("Minutes", colors, "minutes"))

colored_markers = (sf_stations_map_data
      .join('labels', colors_cc, 'End Station')
      .select('lat', 'long', 'labels', 'colors'))
Marker.map_table(colored_markers)

In [ ]:
station_starts = stations.join('name', starts, 'Start Station')
station_starts.show(3)

In [ ]:
sf_starts = station_starts.relabeled(
    'name', 'labels').where('landmark', 'San Francisco')

In [ ]:
sf_starts_map_data = sf_starts.select('lat', 'long', 'labels').with_columns(
    'colors', 'blue',
    'areas', sf_starts.column('count') / 10
)
sf_starts_map_data.show(3)

In [ ]:
Circle.map_table(sf_starts_map_data)

## Part 2 - Conditionals and Iteration ##

Now I'll introduce boolean expressions, conditional, and for loops.

## Comparison ##

We have already seen basic boolean expressions before

In [ ]:
3 > 1

In [ ]:
type(3 > 1)

In [ ]:
type(True)

In [ ]:
true

(We see that capitalization matters.)

Recall that the single `=` is assignment Thus the following error:

In [ ]:
3 = 3

Equality:

In [ ]:
3 == 3.0

Inequality (not equal to):

In [ ]:
10 != 2

Using variables in boolean expressions:

In [ ]:
x = 14
y = 3

In [ ]:
x > 15

In [ ]:
12 < x

In [ ]:
x < 20

Compound boolean expressions:

In [ ]:
12 < x < 20

In [ ]:
(12 < x) and (x < 20)

In [ ]:
(12 < x) and (x < 5)

In [ ]:
(12 < x) or (x < 5)

In [ ]:
10 < x - y < 13

In [ ]:
x > 13 and y < 3.14159

(The comparison `12 < x < 20` is equivalent to `12 < x and x < 20`.)

## Comparisons with arrays

In [ ]:
# An array of pets
pets = make_array('cat', 'cat', 'dog', 'cat', 'dog', 'rabbit')

In [ ]:
# Here it checks whether each element in the array is equal (==) to cat
pets == 'cat'

How many cats?

In [ ]:
sum(pets == 'cat')

Math with booleans

In [ ]:
# Converts to true and false and then counts them as one or zero
np.mean(pets == 'cat')

In [ ]:
# An assignment statement with equality operation
is_cat = pets == 'cat'
is_cat

In [ ]:
is_cat * 2 - 1

In [ ]:
sum(make_array(True, True, False, True, False, False))

In [ ]:
np.count_nonzero(pets == 'dog')

In [ ]:
x = np.arange(20, 31)
x

In [ ]:
x > 28

In [ ]:
sum(x > 28)

## Rows & Apply

In [ ]:
# Read in data table
survey = Table.read_table('welcome_survey_sp22.csv')
survey.show(3)

In [ ]:
# first row
survey.row(0)

In [ ]:
r = survey.row(0)

In [ ]:
type(r)

In [ ]:
r.item(0)

Getting a field from a row

In [ ]:
r.item('Year')

In [ ]:
r.item(2)

## Math on Rows

In [ ]:
r2 = survey.select('Extroversion', 'Number of textees', 'Hours of sleep').row(1)
r2

We can apply aggregation functions to that row

In [ ]:
sum(r2)

Recall that if we wanted to **apply** a function to all the rows of a table we use `apply`

In [ ]:
survey.select('Extroversion', 'Number of textees', 'Hours of sleep')

In [ ]:
survey.select('Extroversion', 'Number of textees', 'Hours of sleep').apply(sum)

Let's use this to improve our pivot table

In [ ]:
p = survey.pivot('Sleep position', 'Hours of sleep')
p.show()

Add row totals to the table for the values of sleep positions

In [ ]:
# Add total column to table
p.with_column('Total', p.drop('Hours of sleep').apply(sum))

Why is `Hours of sleep` still in the table?

Do the same thing with a `group` and a `join`:

In [ ]:
p.join('Hours of sleep', survey.group('Hours of sleep')).show()

## Conditional Statements

Conditional statements in python allow us to do different things based on the values in our data

In [ ]:
x = 20

If the value of x is greater than or equal to 18 then print 'You can legally vote'

In [ ]:
# What happens if x changes?
if x >= 18:
    print('You can legally vote.')

Notice that if the boolean expression is False then the body of the if statement is not executed:



In [ ]:
print("Can you drink?")

if x >= 21:
    print('You can legally drink.')
    print("This line of code is never run...")
    x = 0

print("This is run")
print("The value of x is", x)

Sometimes you want to do something else if hte first statement wasn't true:

In [ ]:
if x >= 21:
    print('You can legally vote and drink.')
else:
    print('You can legally drink milk.')

In [ ]:
if x >= 21:
    print('You can legally vote and drink.')
elif x >= 18:
    print('You can legally vote.')
else:
    print('You can legally drink milk.')

Implementing a function with conditionals and multiple return values:

In [ ]:
def age(x):
    if x >= 18:
        return 'You can legally vote.'
    if x >= 21:
        return 'You can legally drink.'

In [ ]:
age(3)

In [ ]:
age(20)

In [ ]:
age(25)

In [ ]:
def age(x):
    if x >= 21:
        return 'You can legally vote and drink.'
    elif x >= 18:
        return 'You can legally vote.'
    else:
        return 'You can legally drink milk.'

In [ ]:
age(3)

In [ ]:
age(20)

In [ ]:
age(23)

## Putting the pieces together <br>
Here we will build a function that returns whether the trip was one day or a round trip

In [ ]:
# Bike sharing dataset (already have from above, reminder)
trip = Table().read_table('trip.csv')
trip.show(3)

In [ ]:
# Define a function to determine the kind of trip
def trip_kind(start, end):
    if start == end:
        return 'round trip'
    else:
        return 'one way'

In [ ]:
# Add a column of the kind of trip to the table by using apply and our new function to the rows of data
kinds = trip.with_column('Trip Kind', trip.apply(trip_kind, 'Start Station', 'End Station'))
kinds.show(3)

In [ ]:
# Now create a pivot table
kinds_pivot = kinds.where('Duration', are.below(600)).pivot('Trip Kind', 'Start Station').sort('round trip', descending = True).take(np.arange(10))
kinds_pivot

## Simulation

We will use simulation heavily in this class. A key element of simulation is leveraging randomness. The numpy python library has many functions for generating random events. Today we will use the `np.random.choice` function:

### Random Selection

In [ ]:
mornings = make_array('wake up', 'sleep in')

In [ ]:
np.random.choice(mornings)

In [ ]:
np.random.choice(mornings)

In [ ]:
np.random.choice(mornings)

We can also pass an argument that specifies how many times to make a random choice:

In [ ]:
np.random.choice(mornings, 7)

In [ ]:
morning_week = np.random.choice(mornings, 7)
morning_week

In [ ]:
sum(morning_week == 'wake up')

In [ ]:
sum(morning_week == 'sleep in')

## Playing a Game of Chance

Let's play a game: we each roll a die.

If my number is bigger: you pay me a dollar.

If they're the same: we do nothing.

If your number is bigger: I pay you a dollar.

Steps:
1. Find a way to simulate two dice rolls.
2. Compute how much money we win/lose based on the result.
3. Do steps 1 and 2 10,000 times.

### Simulating the roll of a die

In [ ]:
die_faces = np.arange(1, 7)
die_faces

In [ ]:
np.random.choice(die_faces)

Implement a function to simulate a single round of play and return the results

In [ ]:
def simulate_one_round():
    my_roll = np.random.choice(die_faces)
    your_roll = np.random.choice(die_faces)

# finish function

In [ ]:
simulate_one_round()

### Appending Arrays

Sometimes we will want to collect the outcomes of our simulations into a single array. We can do this by appending each experiment to the end of an array using the numpy `np.append` function.

In [ ]:
# Setup two arrays
first = np.arange(4)
second = np.arange(10, 17)

In [ ]:
# Add on to the first array
np.append(first, 6)

In [ ]:
# The first array doesn't change
first

In [ ]:
# Make a longer array
np.append(first, second)

In [ ]:
# Again, first doesn't change
first

In [ ]:
# And second doesn't change
second

### Repeated Betting ###

In [ ]:
results = make_array()

In [ ]:
results = np.append(results, simulate_one_round())
results

In [ ]:
results = np.append(results, simulate_one_round())
results

## `For` Statements

The for statement is another way to apply code to each element in a list or an array.



In [ ]:
for pet in make_array('cat', 'dog', 'rabbit'):
    print('I love my ' + pet)

In [ ]:
pet = make_array('cat', 'dog', 'rabbit').item(0)
print('I love my ' + pet)

pet = make_array('cat', 'dog', 'rabbit').item(1)
print('I love my ' + pet)

pet = make_array('cat', 'dog', 'rabbit').item(2)
print('I love my ' + pet)

What is the output of this for loop?

In [ ]:
x = 0
for i in np.arange(1,4):
  x = x + i
  print(x)

print('The final value of x is:', x)

Use a for loop to simulate the total outcome of plays of our game of chance:

In [ ]:
game_outcomes = make_array()

for i in np.arange(5):
    game_outcomes = np.append(game_outcomes, simulate_one_round())

game_outcomes

In [ ]:
game_outcomes = make_array()

for i in np.arange(10000):
    game_outcomes = np.append(game_outcomes, simulate_one_round())

game_outcomes

In [ ]:
len(game_outcomes)

In [ ]:
results = Table().with_column('My winnings', game_outcomes)

In [ ]:
results

In [ ]:
results.group('My winnings').barh('My winnings')

### Another example: simulating heads in 100 coin tosses

In [ ]:
# Example coin flip
coin = make_array('heads', 'tails')

In [ ]:
# Use np.random.choice to pick
np.random.choice(coin)

In [ ]:
# Pick multiple times
ten_picks = np.random.choice(coin, 10)
ten_picks

In [ ]:
ten_picks == 'heads'

In [ ]:
sum(ten_picks == 'heads')

In [ ]:
# Count the sum of heads coin flip results in our array
sum(np.random.choice(coin, 10) == 'heads')

In [ ]:
sum(np.random.choice(coin, 100) == 'heads')

In [ ]:
# Simulate one outcome

def num_heads():
    return sum(np.random.choice(coin, 100) == 'heads')

In [ ]:
# Decide how many times you want to repeat the experiment
repetitions = 10000

In [ ]:
# Simulate that many outcomes
outcomes = make_array()

for i in np.arange(repetitions):
    outcomes = np.append(outcomes, num_heads())

In [ ]:
heads = Table().with_column('Heads', outcomes)
heads.hist(bins = np.arange(29.5, 70.6))

## Optional: Advanced `where` ##

In [ ]:
ages = make_array(16, 22, 18, 15, 19, 15, 16, 21)
age = Table().with_column('Age', ages)

In [ ]:
age

In [ ]:
age.where('Age', are.above_or_equal_to(18))

In [ ]:
voter = ages >= 18

In [ ]:
voter

In [ ]:
age.where(voter)

In [ ]:
is_voter = are.above_or_equal_to(18)

In [ ]:
type(is_voter)

In [ ]:
is_voter(22)

In [ ]:
is_voter(3)

In [ ]:
age.apply(is_voter, 'Age')

In [ ]:
ages >= 18

In [ ]:
voter

In [ ]:
def my_voter_function(x):
    return x >= 18

In [ ]:
age.where('Age', are.above_or_equal_to(18))

In [ ]:
age.where(voter)

In [ ]:
age.where('Age', my_voter_function)